In [1]:
# webdriver-manager 불러오기
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()

'/Users/mac/.wdm/drivers/chromedriver/mac64/121.0.6167.85/chromedriver-mac-x64/chromedriver'

#### chromedriver 실행이 되지 않을경우 경로지정: https://emessell.tistory.com/entry/PYTHON-selenium-webdriver-%EA%B2%BD%EB%A1%9C-%EC%B2%98%EB%A6%AC

- (1) 상대경로
path = "./chromedriver"

- (2) 절대경로
import os
path = os.getcwd() + "/chromedriver"

- browser = webdriver.Chrome() # 경로지정
- browser.get("http://naver.com")

# 기본함수
- browser.back() # 뒤로가기
- browser.forward() # 앞으로가기
- browser.refresh() # 새로고침

# 문법
- By.CLASS_NAME	태그의 클래스명으로 추출
- By.TAG_NAME	태그 이름으로 추출
- By.ID	태그의 id값으로 추출
- By.XPATH	태그의 경로로 추출

# check if class name exists or not
- https://www.geeksforgeeks.org/how-to-check-if-an-element-exists-with-python-selenium/

In [2]:
# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

import time
from datetime import datetime
import random
import re

In [3]:
# 브라우저 띄우기
options = Options()
user = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
options.add_argument(f"User-Agent = {user}")
browser = webdriver.Chrome(options=options)

link_list = []
for page in range(1,4):
    print("*"*10, f'현재 {page} 페이지 수집중입니다.', "*"*10)
    url = "https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={}&pageSize=24".format(page)
    
    # 브라우저 띄우기
    browser.get(url)
     # 각 페이지마다 도서의 링크 데이터 수집
    data = browser.find_elements(By.CLASS_NAME, 'gd_name')
    for i in data:
        link = i.get_attribute("href")
        link_list.append(link)

    time.sleep(3)

print(link_list)

********** 현재 1 페이지 수집중입니다. **********
********** 현재 2 페이지 수집중입니다. **********
********** 현재 3 페이지 수집중입니다. **********
['https://www.yes24.com/Product/Goods/122120495', 'https://www.yes24.com/Product/Goods/123675187', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/124043812', 'https://www.yes24.com/Product/Goods/124702623', 'https://www.yes24.com/Product/Goods/124043802', 'https://www.yes24.com/Product/Goods/122090360', 'https://www.yes24.com/Product/Goods/123400303', 'https://www.yes24.com/Product/Goods/122944685', 'https://www.yes24.com/Product/Goods/118579613', 'https://www.yes24.com/Product/Goods/124539805', 'https://www.yes24.com/Product/Goods/124110821', 'https://www.yes24.com/Product/Goods/124112599', 'https://www.yes24.com/Product/Goods/122428190', 'https://www.yes24.com/Product/Goods/118413014', 'https://www.yes24.com/Product/Goods/119458052', 'https://www.yes24.com/Product/Goods/119562091', 'https://www.yes24.com/Product/Goods/124702605', 

In [4]:
print(len(link_list))

72


In [5]:
# 데이터베이스 연동 후 수집한 데이터 DB 저장(csv)
import pymysql

connection = pymysql.connect(
host = 'localhost',
user = 'root',
password = 'kr14021428',
db = 'yes24',
charset = 'utf8mb4',
cursorclass = pymysql.cursors.DictCursor
)

with connection.cursor() as cursor:
    # 상세페이지 이동후 크롤링
    for link in link_list:
        browser.get(link)

        title = browser.find_element(By.CLASS_NAME, "gd_name").text
        # author
        author = browser.find_element(By.CLASS_NAME, "gd_auth").text
        # publisher
        publisher = browser.find_element(By.CLASS_NAME, "gd_pub").text
        # publishing: : 2024년 1월 1일 -> 2024-01-01(type: Date) ->2024-01-01(type: str) 
        publishing = browser.find_element(By.CLASS_NAME,"gd_date").text
        publishing = datetime.strptime(publishing, '%Y년 %m월 %d일').date().strftime("%Y-%m-%d")
        """        
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
        if match:
            year, month, day = match.groups()
            date_obj = datetime(int(year), int(month), int(day))
            publishing = date_obj.strftime("%Y-%m-%d")
        else:
            publishing = "2023-01-01"
        """
        # rating
        rating_area = browser.find_element(By.CLASS_NAME,"gd_ratingArea").text
        if rating_area.startswith("리뷰"):
            rating = browser.find_element(By.CLASS_NAME, "yes_b").text
        else: # startwith 첫번째
            rating = 0
        # review
        review = browser.find_element(By.CLASS_NAME,"gd_reviewCount").text.replace(",","")
        if review == "첫번째 리뷰어가 되어주세요.":
            review = 0
        else: 
            review = "".join(filter(str.isdigit, review))
        # sales
        sales = browser.find_element(By.CLASS_NAME,"gd_sellNum").text.split(" ")[2].replace(",","")
        # price
        price = browser.find_element(By.CLASS_NAME,"yes_m").text[:-1].replace(",","")
        # ranking
        rank_text = browser.find_element(By.CLASS_NAME,"gd_best").find_element(By.TAG_NAME, "dd").text.split(" ")
        ranking = rank_text[1][:-1]
        # ranking_weeks
        ranking_weeks = rank_text[-1][:-1]
        '''
        rank_text = browser.find_element(By.CLASS_NAME,"gd_best").text.split("|")
        print(rank_text)
        # 숫자만 추출
        # str.isdigit(): 숫자로만 이루어진 str판별해주는 함수 true or false 반환
        ranking = "".join(filter(str.isdigit, rank_text[0])) 
        ranking_weeks = "".join(filter(str.isdigit, rank_text[1].split(" ")[-1]))
        print(ranking)
        print(ranking_weeks)
        '''

        sql = '''
        insert into Books(title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks) 
        values (%s, %s, %s, %s,%s, %s, %s, %s,%s, %s)
        '''
        cursor.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
        time.sleep(2)
    connection.commit()

<>:26: SyntaxWarning: invalid escape sequence '\d'
<>:26: SyntaxWarning: invalid escape sequence '\d'
/var/folders/kt/8mj6y1rn443fz0mmq4th9cn40000gn/T/ipykernel_3608/2993924114.py:26: SyntaxWarning: invalid escape sequence '\d'
  """
